In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
!pip install transformers
from transformers import BertTokenizer,BertModel
import torch.nn.functional as F
import torch.optim as optim

     |████████████████████████████████| 665kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 11.2MB/s 
     |████████████████████████████████| 3.8MB 323kB/s 
     |████████████████████████████████| 890kB 35.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=afd3bb1d4c6a7a60694c4114ab1b17900484e7f56b30a5e9d92f0dfc0e3b76da
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
df1=pd.read_csv('https://raw.githubusercontent.com/rachitban/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_data_all.csv',sep=',')
df1

,id,sent0,sent1
0,0,He poured orange juice on his cereal.,He poured milk on his cereal.
1,1,He drinks apple.,He drinks milk.
2,2,Jeff ran a mile today,"Jeff ran 100,000 miles today"
3,3,A mosquito stings me,I sting a mosquito
4,4,A niece is a person.,A giraffe is a person.
...,...,...,...
9995,9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie
9996,9996,Gloria wears a cat on her head,Gloria wears a hat on her head
9997,9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...
9998,9998,Reilly is sleeping on the couch,Reilly is sleeping on the window


In [ ]:
df2=pd.read_csv('https://raw.githubusercontent.com/rachitban/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_answers_all.csv',sep=',',names=['id','ans'])
df2

,id,ans
0,0,0
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
9995,9995,0
9996,9996,0
9997,9997,1
9998,9998,1


In [ ]:
df2=df2['ans']
main_df=pd.concat([df1,df2],axis=1)

In [ ]:
main_df

,id,sent0,sent1,ans
0,0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,1,He drinks apple.,He drinks milk.,0
2,2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1
3,3,A mosquito stings me,I sting a mosquito,1
4,4,A niece is a person.,A giraffe is a person.,1
...,...,...,...,...
9995,9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie,0
9996,9996,Gloria wears a cat on her head,Gloria wears a hat on her head,0
9997,9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,1
9998,9998,Reilly is sleeping on the couch,Reilly is sleeping on the window,1


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0") 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [ ]:
class MyClass(nn.Module):
  def __init__(self):
    super(MyClass,self).__init__()

    self.fc1=nn.Linear(1024,20)
    self.fc2=nn.Linear(20,1)
    self.Relu=F.relu

  def forward(self,sentence1,sentence2):
    
    output1=self.fc1(sentence1)
    output1=self.Relu(output1)
    output1=self.fc2(output1)
    
    output2=self.fc1(sentence2)
    output2=self.Relu(output2)
    output2=self.fc2(output2)

    a1=torch.exp(output1)/(torch.exp(output1) + torch.exp(output2))
    a2=torch.exp(output2)/(torch.exp(output1) + torch.exp(output2))
    
    #print(a1,a2)

    return a1,a2



In [ ]:
model=MyClass()
# model.load_state_dict(torch.load('taskA_2.pt')) # comment this in case model is not saved 1st time
model.to(device)

loss_fct=nn.MarginRankingLoss(margin=0.4)
optimizer=optim.Adam(model.parameters())

In [ ]:
train_df=main_df[0:1000]
train_df

test_df=main_df[9000:10000]

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-large-uncased')
bertmodel=BertModel.from_pretrained('bert-large-uncased')
bertmodel.to(device)
bertmodel.eval()
model.train()

a=[]
b=[]

for epoch in range (1):
  j=0
  l1=0
  correct=0
  for i in train_df.values:

    tokenize_input=tokenizer.tokenize(i[1].lower())
    sentence1=torch.tensor([tokenizer.encode(tokenize_input,add_special_tokens=True)])
    sentence1=bertmodel(sentence1.to(device))
    
    tokenize_input=tokenizer.tokenize(i[2].lower())
    sentence2=torch.tensor([tokenizer.encode(tokenize_input,add_special_tokens=True)])
    sentence2=bertmodel(sentence2.to(device))
  
    if(i[3]==0):
      y=torch.FloatTensor([-1.0]).to(device)
    else:
      y=torch.FloatTensor([1.0]).to(device)

    y1,y2=model(sentence1[0][0][0],sentence2[0][0][0])
  
    loss=loss_fct(y1,y2,y)
    # print(j,loss.data)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    l1+=loss
    j+=1
    print(epoch,j)

    if(y1>=y2 and i[3]==1):
      correct+=1
    elif(y1<y2 and i[3]==0):
      correct+=1

  print(correct)
  a.append(l1/j)
  b.append(epoch+1)
  
  plt.plot(b,a,'g')
  plt.show()

# plt.show()
    

In [ ]:
torch.save(model.state_dict(),'taskA_2.pt')

In [ ]:
model.eval()
correct=0

for i in test_df.values:

    tokenize_input=tokenizer.tokenize(i[1].lower())
    sentence1=torch.tensor([tokenizer.encode(tokenize_input,add_special_tokens=True)])
    sentence1=bertmodel(sentence1.to(device))
    
    tokenize_input=tokenizer.tokenize(i[2].lower())
    sentence2=torch.tensor([tokenizer.encode(tokenize_input,add_special_tokens=True)])
    sentence2=bertmodel(sentence2.to(device))

    y1,y2=model(sentence1[0][0][0].to(device),sentence2[0][0][0].to(device))

    if(y1>y2 and i[3]==1):
      correct+=1
    elif(y1<y2 and i[3]==0):
      correct+=1
    # print(y1,y2)

print(correct)

In [ ]:
test_df

In [ ]:
# 1424 , 1439 , 1426 , 1424 , 1415 for 768 x 20
# 1492 , 1423 